In [2]:
!wget https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/master/advanced_functionality/tensorflow_bring_your_own/utils/generate_cifar10_tfrecords.py
!pip install ipywidgets
!python generate_cifar10_tfrecords.py --data-dir cifar10

--2022-01-12 23:53:51--  https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/tensorflow_bring_your_own/utils/generate_cifar10_tfrecords.py
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘generate_cifar10_tfrecords.py.1’

generate_cifar10_tf     [ <=>                ] 230.24K  --.-KB/s    in 0.07s   

2022-01-12 23:53:52 (3.46 MB/s) - ‘generate_cifar10_tfrecords.py.1’ saved [235770]



In [ ]:
import time, os, sys
import sagemaker, boto3
import numpy as np
import pandas as pd

sess = boto3.Session()
sm   = sess.client('sagemaker')
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)

datasets = sagemaker_session.upload_data(path='cifar10', key_prefix='datasets/cifar10-dataset')
datasets 

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

training_experiment = Experiment.create(
                                experiment_name = "sagemaker-training-experiments-3", 
                                description     = "Experiment to track cifar10 training trials", 
                                sagemaker_boto_client=sm)

In [ ]:
hvd_instance_type = 'ml.p3.16xlarge'
hvd_instance_count = 2
hvd_processes_per_host = 1

print(f'Distributed training with a total of {hvd_processes_per_host*hvd_instance_count} workers: {hvd_instance_count} instances of {hvd_instance_type}')
print(f'{hvd_processes_per_host} GPU(s) per instance')

In [ ]:
distributions = {
                 'mpi': {
                          'enabled'           : True,
                          'processes_per_host': hvd_processes_per_host,
                          'custom_mpi_options': '-verbose --NCCL_DEBUG=INFO -x OMPI_MCA_btl_vader_single_copy_mechanism=none'
                        }
                }

In [ ]:
from sagemaker.tensorflow import TensorFlow

hyperparams={'epochs'       : 30,
             'learning-rate': 0.01,
             'batch-size'   : 256,
             'weight-decay' : 2e-4,
             'momentum'     : 0.9,
             'optimizer'    : 'adam'}

bucket_name = sagemaker_session.default_bucket()
output_path = f's3://{bucket_name}/jobs'
metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

tf_estimator = TensorFlow(entry_point          = 'cifar10-tf-horovod-sagemaker.py', 
                          output_path          = f'{output_path}/',
                          code_location        = output_path,
                          role                 = role,
                          train_instance_count = hvd_instance_count, 
                          train_instance_type  = 'ml.g4dn.xlarge',
                          train_volume_size    = 50,
                          framework_version    = '1.15.2', 
                          py_version           = 'py3',
                          script_mode          = True,
                          metric_definitions   = metric_definitions,
                          sagemaker_session    = sagemaker_session,
                          hyperparameters      = hyperparams,
                          distributions        = distributions)

job_name=f'tensorflow-single-gpu-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'
tf_estimator.fit({'training'  : datasets,
                  'validation': datasets,
                  'eval'      : datasets},
                 job_name = job_name,
                 experiment_config=experiment_config)